In [38]:
%load_ext autoreload
%autoreload 2

import sys, os, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
print(results_df_old.shape)
print(results_df_old.columns)
results_df_old.head()

(41, 39)
Index(['run_id', 'lr', 'net', 'bias', 'name', 'seed', 'tags', 'test', 'n_val',
       'train', 'y_low', 'device', 'n_test', 'y_high', 'log_dir', 'n_train',
       'predict', 'version', 'data_dir', 'eps_mult', 'n_splits', 'do_return',
       'noise_std', 'only_flip', 'batch_size', 'max_epochs', 'data_kwargs',
       'num_workers', 'space_length', 'generate_data', 'model_summary',
       'args_processed', 'do_return_model', 'transform_kwargs',
       'persistent_workers', 'run_name', 'test_loss', 'logger', 'grid_size'],
      dtype='object')


,run_id,lr,net,bias,name,seed,tags,test,n_val,train,...,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss,logger,grid_size
0,2m7eu5tn,0.0010,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,worldly-snowflake-183,[0.0004728641652036458],NaN,NaN
1,ugylvzz6,0.0001,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,classic-glade-184,[0.009488677605986595],NaN,NaN
2,unpw27ml,0.1000,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,sage-river-182,[0.0035890007857233286],NaN,NaN
3,w4yjcu25,0.0100,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,azure-grass-181,[0.0031849860679358244],NaN,NaN
4,q86f7s5e,0.0001,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,blooming-star-180,[0.02364257164299488],NaN,NaN


In [44]:
api = wandb.Api()
filters = {'state': 'finished'}
runs = api.runs('eliasdubbeldam/symlie', filters=filters)
run_id_api = {run.id for run in runs}

In [52]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

run_dirs =  np.array([[run_dir.split('-')[-1], run_dir] for run_dir in os.listdir(wandb_dir) if run_dir.startswith('run')])
if not len(run_dirs) == 0:
    assert len(run_dirs[:, 0]) == len(set(run_dirs[:, 0]))
    run_dirs = {run_id: run_dir for run_id, run_dir in run_dirs}
    run_dirs_df = pd.DataFrame(run_dirs.items(), columns = ['run_id', 'run_dir']).sort_values(by = 'run_dir', ascending = False)
else:
    run_dirs_df = pd.DataFrame(columns = ['run_id', 'run_dir'])


for i, values in run_dirs_df.iterrows():
    run_id = values['run_id']
    if run_id not in run_id_api:
        print(f'Removing {run_id}')
        # shutil.rmtree(os.path.join(wandb_dir, values['run_dir']), )

run_dirs_df

,run_id,run_dir
0,ikydo5s3,run-20240129_142142-ikydo5s3


In [53]:
Ps = os.listdir(os.path.join(store_dir, 'P'))
for P in Ps:
    P_id = P.split('.')[0]
    if P_id not in run_id_api:
        print(f'Removing {P_id}')
        os.remove(os.path.join(store_dir, 'P', P))

In [9]:
exclude_ids = []

In [14]:
tags = 'dev'

api = wandb.Api()
filters = {'tags': tags} if tags else {}
filters['state'] = 'finished'
runs = api.runs('eliasdubbeldam/symlie', filters=filters)
print(len(runs))
for i, run in enumerate(runs):
    print(run.name, run.id, run.tags)
    # if run.id in exclude_ids: run.delete()

1
zany-dawn-207 ikydo5s3 ['dev', 'sine1d']


In [15]:
listdir_dirs = lambda path: [entry for entry in os.listdir(path) if os.path.isdir(os.path.join(path, entry))]
store_dir_dirs = listdir_dirs(store_dir)
store_dir_dirs

['P']

In [17]:
config_list = []

pbar = tqdm(runs)
for run in pbar:
    pbar.set_description(f'Retreiving wandb {str(id)}')
    
    id = run.id

    # Exceptions
    if run.state != 'finished':
        continue
    if id in results_df_old['run_id'].values:
        continue
    if id in exclude_ids:
        continue

    # print(id)
    # continue

    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Retreive metrics from wandb run
    history = run.history()
    metrics = ['test_loss'] # 'train_loss', 'val_loss'
    for metric in metrics:
        config[metric] = np.array(history[metric].dropna())

    # Retreive arrays from wandb files
    # npu = NumpyUtils(dir = os.path.join(wandb_dir, run_dirs[id], 'files'))
    # run_arrays = npu.load_all()
    # for filename, array in run_arrays.items():
    #     config[filename] = array
    
    # for store_dir_dir in store_dir_dirs:
    #     array = np.load(os.path.join(store_dir, store_dir_dir, f'{id}.npy'))    
    #     config[store_dir_dir] = array


    config_list.append(config)

results_df = pd.DataFrame(config_list)
results_df

Retreiving wandb ikydo5s3: 100%|██████████| 1/1 [00:04<00:00,  4.41s/it]


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,data_kwargs,num_workers,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss
0,0.1,TrainP,False,None,42,"[dev, sine1d]",True,1000,True,1,...,"{'y_low': 1, 'y_high': 3, 'grid_size': [1, 7],...",7,False,False,True,False,"{'eps_mult': [0, 0, 1, 0], 'only_flip': False}",True,zany-dawn-207,[6.171144195832312e-05]


In [18]:
results_df_new = pd.concat([results_df_old, results_df])
results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))